In [1]:
!pip install riotwatcher

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

In [11]:
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

import sys,os
file_path = os.path.join(sys.path[0], ".secret/riotapi.json")

keys = get_keys(file_path)
API_Key = keys['API_Key']


In [12]:
from riotwatcher import LolWatcher, ApiError

lol_watcher = LolWatcher(API_Key)

my_region = 'na1'

me = lol_watcher.summoner.by_name(my_region, 'thereiswald0')
print(me)

SSLError: HTTPSConnectionPool(host='na1.api.riotgames.com', port=443): Max retries exceeded with url: /lol/summoner/v4/summoners/by-name/thereiswald0 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [ ]:
# all objects are returned (by default) as a dict
# lets see if i got diamond yet (i probably didnt)
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

In [ ]:
# First we get the latest version of the game from data dragon
versions = lol_watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']

# Lets get some champions
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
print(current_champ_list)


In [ ]:
current_champ_list['data']['Soraka']

In [ ]:
try:
    response = lol_watcher.summoner.by_name(my_region, 'this_is_probably_not_anyones_summoner_name')
except ApiError as err:
    if err.response.status_code == 429:
        print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
        print('this retry-after is handled by default by the RiotWatcher library')
        print('future requests wait until the retry-after time passes')
    elif err.response.status_code == 404:
        print('Summoner with that ridiculous name not found.')
    else:
        raise

Show the main champions that we all use and how they rank in each category. Then do simulations to help explain ap/mr, ad/armour. Finally show the effect of items. Item spike in damage, item percentage of stats at different levels, fully spec'd out what does tha mean.

In [ ]:
my_matches = lol_watcher.match.matchlist_by_account(my_region, me['accountId'])

# fetch last match detail
last_match = my_matches['matches'][0]
match_detail = lol_watcher.match.by_id(my_region, last_match['gameId'])

participants = []
for row in match_detail['participants']:
    participants_row = {}
    participants_row['champion'] = row['championId']
    participants_row['spell1'] = row['spell1Id']
    participants_row['spell2'] = row['spell2Id']
    participants_row['win'] = row['stats']['win']
    participants_row['kills'] = row['stats']['kills']
    participants_row['deaths'] = row['stats']['deaths']
    participants_row['assists'] = row['stats']['assists']
    participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
    participants_row['goldEarned'] = row['stats']['goldEarned']
    participants_row['champLevel'] = row['stats']['champLevel']
    participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
    participants_row['item0'] = row['stats']['item0']
    participants_row['item1'] = row['stats']['item1']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

In [ ]:
# check league's latest version
latest = lol_watcher.data_dragon.versions_for_region(my_region)['n']['champion']
# Lets get some champions static information
static_champ_list = lol_watcher.data_dragon.champions(latest, False, 'en_US')

# champ static list data to dict for looking up
champ_dict = {}
for key in static_champ_list['data']:
    row = static_champ_list['data'][key]
    champ_dict[row['key']] = row['id']
for row in participants:
    print(str(row['champion']) + ' ' + champ_dict[str(row['champion'])])
    row['championName'] = champ_dict[str(row['champion'])]

# print dataframe
df = pd.DataFrame(participants)
df

In [ ]:
champs = pd.DataFrame.from_dict(static_champ_list['data'])

In [ ]:
champs.loc['stats']['Lux']

In [ ]:
champs

In [ ]:
champ_stats = pd.DataFrame()
for champ in list(champs.columns):
#     champ_stats[champ] = champs.loc['stats'][champ]
#     champ_stats.index = list(champs.loc['stats'][champ].keys())
    champ_stats[champ] = champs.loc['stats'][champ].values()
champ_stats = champ_stats.set_index([pd.Index(list(champs.loc['stats'][champ].keys()))])
champ_stats = champ_stats.T # transpose so it's easier to manipulate the data
champ_stats = champ_stats.sort_values(by=['hp'], ascending=False)
champ_stats['hp'].plot()
# champ_stats

In [ ]:
# champ_stats.loc['hp'].plot()
champ_stats.T.sort_values(by=['attackdamage'], ascending=False)

In [ ]:
static_champ_list